In [ ]:
%pip install --upgrade pip

# Uninstall conflicting packages
%pip uninstall -y langchain_classic langchain-core langchain-openai langchain-community langchain langchain-chroma chromadb beautifulsoup4 python-dotenv PyPDF2 rank_bm25 weaviate-client ragas wikipedia langchain-weaviate langchain-together gradio 6.0.2

# Step 2.3: Install required packages
%pip install neo4j==6.0.3
%pip install rdflib==7.5.0
%pip install pandas==2.3.3
%pip install sentence-transformers==5.1.2
%pip install faiss-cpu==1.13.1
%pip install python-dotenv==1.2.1
%pip install langchain==1.1.0
%pip install langchain-openai==1.1.0
%pip install langchain-community==0.4.1

In [ ]:
# Step 3.2: Set up imports
import os
import rdflib
from rdflib.namespace import RDF, RDFS, OWL
import pandas as pd
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Any
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import textwrap

# Load env vars from the file used in previous chapters
_ = load_dotenv(dotenv_path='env.txt')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Neo4j setup
NEO4J_URI = os.getenv('NEO4J_URI', 'neo4j://127.0.0.1:7687')
NEO4J_USER = os.getenv('NEO4J_USER', 'neo4j')
NEO4J_PASS = os.getenv('NEO4J_PASS', 'password')

# LLM setup
CHAT_MODEL = "gpt-4o-mini"
llm = ChatOpenAI(model=CHAT_MODEL, temperature=0.2)
os.environ["LANGCHAIN_TRACING_V2"] = "false"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))

def reset_neo4j_database():
    """Remove all nodes and relationships from Neo4j"""
    with driver.session() as session:
        result = session.run("MATCH (n) DETACH DELETE n")
        summary = result.consume()
        print(f"Neo4j database reset - deleted {summary.counters.nodes_deleted} nodes and {summary.counters.relationships_deleted} relationships")

# --- Test connection and optionally reset ---
try:
    with driver.session() as session:
        session.run("RETURN 1")
    print(f"Connected to Neo4j at {NEO4J_URI} as user {NEO4J_USER}")
    
    # Uncomment the line below to reset your database
    # WARNING: This will delete ALL data in your Neo4j instance
    # reset_neo4j_database()
    
except Exception as e:
    print(f"WARNING: Could not connect to Neo4j at {NEO4J_URI}")
    print(f"Make sure Neo4j Desktop is running and the database is started.")
    print(f"Error: {e}")

In [ ]:
# Step 3.3: Load the ontology
g = rdflib.Graph()
g.parse('FinancialOntology.ttl', format='turtle')

# Helper to get first value of a given property
def get_first(subject, prop):
    for val in g.objects(subject, prop):
        return str(val)
    return None

# --- Collect nodes ---
nodes = []
for s in g.subjects(RDF.type, OWL.Class):
    nodes.append({
        'id': str(s),
        'label': get_first(s, RDFS.label) or s.split('#')[-1],
        'comment': get_first(s, RDFS.comment),
        'type': 'Class'
    })
for s in g.subjects(RDF.type, OWL.NamedIndividual):
    nodes.append({
        'id': str(s),
        'label': get_first(s, RDFS.label) or s.split('#')[-1],
        'comment': get_first(s, RDFS.comment),
        'type': 'Individual'
    })
nodes_df = pd.DataFrame(nodes)
nodes_df.to_csv('ontology_nodes.csv', index=False)

# --- Collect edges ---
edges = []
for s, p, o in g.triples((None, None, None)):
    if p in [RDF.type, RDFS.label, RDFS.comment]:
        continue
    if str(p).startswith('http://www.w3.org/2002/07/owl#'):
        continue
    if isinstance(o, rdflib.term.Identifier) and str(o).startswith('http'):
        edges.append({
            'source': str(s),
            'target': str(o),
            'type': p.split('#')[-1] if '#' in str(p) else str(p).split('/')[-1]
        })
edges_df = pd.DataFrame(edges)
edges_df.to_csv('ontology_edges.csv', index=False)

print("Created ontology_nodes.csv and ontology_edges.csv")

data_rows = []
for s, p, o in g.triples((None, None, None)):
    # Keep only literal values (data properties)
    if isinstance(o, rdflib.term.Literal):
        prop_name = p.split('#')[-1] if '#' in str(p) else str(p).rstrip('/').split('/')[-1]
        # capture datatype if present
        dtype = str(o.datatype) if o.datatype else None
        data_rows.append({
            'subject': str(s),
            'property': prop_name,
            'value': str(o),
            'datatype': dtype
        })

pd.DataFrame(data_rows).to_csv('ontology_data.csv', index=False)
print("Created ontology_data.csv")


In [ ]:
# Step 4.1: Use credentials loaded in previous cells
def run_tx(query, params=None):
    with driver.session() as session:
        return session.run(query, params or {}).consume()

print(f"Connected to Neo4j at {NEO4J_URI} as user {NEO4J_USER}")

In [ ]:
# Step 4.2: Create schema constraint
run_tx("""
CREATE CONSTRAINT resource_id_unique IF NOT EXISTS
FOR (n:Resource) REQUIRE n.id IS UNIQUE
""")
print("Constraint ensured: (:Resource {id}) is UNIQUE.")


In [ ]:
# Step 4.3: Load nodes from ontology_nodes.csv
nodes_df = pd.read_csv("ontology_nodes.csv")
print(nodes_df.head())

# MERGE all nodes as :Resource; store label/comment/type for later use
node_query = """
MERGE (n:Resource {id: $id})
SET n.rdfs_label = $rdfs_label,
    n.comment    = $comment,
    n.kind       = $kind   // 'Class' or 'Individual'
"""

with driver.session() as session:
    for rec in nodes_df.to_dict(orient="records"):
        params = {
            "id": rec["id"],
            "rdfs_label": rec.get("label"),
            "comment": rec.get("comment"),
            "kind": rec.get("type")
        }
        session.run(node_query, params)
print(f"Imported {len(nodes_df)} nodes as :Resource.")

In [ ]:
# Step 4.4: Load edges from ontology_edges.csv 

edges_df = pd.read_csv("ontology_edges.csv")
print(edges_df.head())

rel_map = {
    "issuedBy": "ISSUED_BY",
    "isRegulatedBy": "IS_REGULATED_BY",
    "ownedBy": "OWNED_BY",
}

query_issued = """
MATCH (a:Resource {id: $src}), (b:Resource {id: $tgt})
MERGE (a)-[:ISSUED_BY]->(b)
"""

query_regulated = """
MATCH (a:Resource {id: $src}), (b:Resource {id: $tgt})
MERGE (a)-[:IS_REGULATED_BY]->(b)
"""

query_owned = """
MATCH (a:Resource {id: $src}), (b:Resource {id: $tgt})
MERGE (a)-[:OWNED_BY]->(b)
"""

with driver.session() as session:
    count = 0
    for rec in edges_df.to_dict(orient="records"):
        t = str(rec.get("type", "")).strip()
        src = rec.get("source")
        tgt = rec.get("target")
        if t not in rel_map:
            continue  # skip edges we aren't modeling here
        if t == "issuedBy":
            session.run(query_issued, {"src": src, "tgt": tgt})
        elif t == "isRegulatedBy":
            session.run(query_regulated, {"src": src, "tgt": tgt})
        elif t == "ownedBy":
            session.run(query_owned, {"src": src, "tgt": tgt})
        count += 1
print(f"Imported {count} relationships (ISSUED_BY, IS_REGULATED_BY, OWNED_BY).")


In [ ]:
# step 4.5: Load data properties from ontology_data.csv

try:
    data_df = pd.read_csv("ontology_data.csv")
except FileNotFoundError:
    data_df = pd.DataFrame(columns=["subject","property","value","datatype"])

print(data_df.head())

# Only apply properties we care about in this lab (hasTicker)
query_has_ticker = """
MATCH (n:Resource {id: $id})
SET n.hasTicker = $val
"""

with driver.session() as session:
    tick_count = 0
    for rec in data_df.to_dict(orient="records"):
        prop = str(rec.get("property", "")).strip()
        if prop != "hasTicker":
            continue
        session.run(query_has_ticker, {"id": rec.get("subject"), "val": rec.get("value")})
        tick_count += 1

print(f"Set hasTicker on {tick_count} nodes.")

In [ ]:
# step 4.6: Find anything that looks like a stock-ish resource with a ticker
with driver.session() as session:
    result = session.run("""
        MATCH (n:Resource)
        WHERE n.hasTicker IS NOT NULL
        RETURN n.rdfs_label AS label, n.hasTicker AS ticker, n.id AS id
        ORDER BY label
    """)
    rows = result.data()
rows

In [ ]:
# Step 5.1: Build rdf:type (class membership) edges from the Turtle file and push to Neo4j
type_pairs = []  # (individual_iri, class_iri)

# We only want membership of individuals to classes (not "is a Class" or "is a NamedIndividual")
for s, _, o in g.triples((None, RDF.type, None)):
    # skip ontology meta (i.e., don't create type edges for the classes themselves)
    if o in (OWL.Class, OWL.NamedIndividual):
        continue
    # many ontologies also mark classes with RDF.type RDFS.Class
    if o == RDFS.Class:
        continue
    # keep only cases where subject looks like an IRI and object looks like a class IRI present in our graph
    if isinstance(s, rdflib.term.Identifier) and isinstance(o, rdflib.term.Identifier):
        type_pairs.append((str(s), str(o)))

len(type_pairs)

In [ ]:
# Step 5.1 (second code section): Push [:IS_A] edges into Neo4j between the resource (individual) and the class node
with driver.session() as session:
    q = """
    MATCH (a:Resource {id: $sid}), (cls:Resource {id: $cid})
    MERGE (a)-[:IS_A]->(cls)
    """
    count = 0
    for sid, cid in type_pairs:
        session.run(q, {"sid": sid, "cid": cid})
        count += 1
count

In [ ]:
# Step 5.2: Create “All X” concept nodes and wire members

anchors = [
    {"name": "All Stocks",  "class_label": "Stock"},
    {"name": "All Bonds",   "class_label": "Bond"},
    {"name": "All Orgs",    "class_label": "Organization"},
    {"name": "All Regulators", "class_label": "RegulatoryAuthority"},
    # add more as your ontology grows
]

with driver.session() as session:
    for a in anchors:
        # ensure the anchor concept exists
        session.run("""
            MERGE (c:Concept {name: $name})
            ON CREATE SET c.description = $desc
        """, {
            "name": a["name"],
            "desc": f"Anchor node that includes all {a['class_label']} members."
        })
        # clear old INCLUDES from this concept (idempotent refresh)
        session.run("""
            MATCH (c:Concept {name: $name})-[r:INCLUDES]->()
            DELETE r
        """, {"name": a["name"]})
        # connect anchor to all members of the class via IS_A
        session.run("""
            MATCH (c:Concept {name: $name})
            MATCH (cls:Resource {rdfs_label: $class_label})
            MATCH (n:Resource)-[:IS_A]->(cls)
            MERGE (c)-[:INCLUDES]->(n)
        """, {"name": a["name"], "class_label": a["class_label"]})

print("Anchors created/updated and INCLUDES relationships established.")

In [ ]:
# Step 5.3 (part 1): Count included members per anchor
with driver.session() as session:
    data = session.run("""
        MATCH (c:Concept)-[:INCLUDES]->(n:Resource)
        RETURN c.name AS anchor, count(n) AS members
        ORDER BY anchor
    """).data()
data

In [ ]:
# Step 5.3 (part 2): Peek at a few Stock members
with driver.session() as session:
    data = session.run("""
        MATCH (:Concept {name:'All Stocks'})-[:INCLUDES]->(n:Resource)
        RETURN n.rdfs_label AS label, n.hasTicker AS ticker
        ORDER BY label
        LIMIT 10
    """).data()
data

In [ ]:
# Step 6.1: Build enriched hybrid text (with multi-hop)
def fetch_hybrid_text_for_class(class_label: str):
    """
    Returns a DataFrame with: id, class_label, hybridText
    for all :Resource nodes that are IS_A the given class_label,
    enriched with multi-hop info.
    """
    cypher = """
    MATCH (n:Resource)-[:IS_A]->(cls:Resource {rdfs_label: $class_label})
    OPTIONAL MATCH (n)-[:ISSUED_BY]->(org:Resource)
    OPTIONAL MATCH (n)-[:IS_REGULATED_BY]->(reg:Resource)
    OPTIONAL MATCH (org)-[:IS_REGULATED_BY]->(orgreg:Resource)
    WITH n, cls, org, reg, orgreg
    RETURN
      n.id AS id,
      cls.rdfs_label AS class_label,
      (
        coalesce(cls.rdfs_label, '') + ' ' +
        coalesce(n.rdfs_label, '') +
        CASE WHEN n.hasTicker IS NOT NULL THEN ' [' + n.hasTicker + ']' ELSE '' END +
        CASE WHEN org.rdfs_label IS NOT NULL THEN ' issued by ' + org.rdfs_label ELSE '' END +
        CASE WHEN reg.rdfs_label IS NOT NULL THEN ' regulated by ' + reg.rdfs_label ELSE '' END +
        CASE WHEN n.comment    IS NOT NULL THEN ' — ' + n.comment ELSE '' END +
        CASE WHEN orgreg.rdfs_label IS NOT NULL THEN '. Issuer regulated by ' + orgreg.rdfs_label ELSE '' END
      ) AS hybridText
    ORDER BY n.rdfs_label
    """
    with driver.session() as session:
        rows = session.run(cypher, {"class_label": class_label}).data()
    return pd.DataFrame(rows)

# Example: build for Stocks and Bonds
stocks_df = fetch_hybrid_text_for_class("Stock")
bonds_df  = fetch_hybrid_text_for_class("Bond")

display(stocks_df.head())
display(bonds_df.head())

In [ ]:
# Step 6.2: Build enriched hybrid text (with multi-hop)
orgs_df  = fetch_hybrid_text_for_class("Organization")
regs_df  = fetch_hybrid_text_for_class("RegulatoryAuthority")

hybrid_df = pd.concat([stocks_df, bonds_df, orgs_df, regs_df], ignore_index=True)
print(f"Total hybrid text entries: {len(hybrid_df)}")
hybrid_df.head(10)

In [ ]:
# Step 6.3: Export for embedding service
out_path = "hybrid_embeddings_input.csv"
hybrid_df.to_csv(out_path, index=False)
print(f"Wrote {len(hybrid_df)} rows to {out_path}")

In [ ]:
# Step 6.4: embeddings + vectorstore (one place only) ---
# local, no-API embedder that returns normalized vectors
class STEmbeddings(Embeddings):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.st = SentenceTransformer(model_name)
    def embed_documents(self, texts):
        return self.st.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return self.st.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

embedder = STEmbeddings()

# Build documents from your hybrid_df (from Step 6.1 / 6.2)
docs = [
    Document(
        page_content=row.hybridText,
        metadata={"id": row.id, "class_label": row.class_label}
    )
    for row in hybrid_df.itertuples(index=False)
]

# Let LangChain build and hold FAISS; no direct `import faiss`
vectorstore = FAISS.from_documents(docs, embedder)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


In [ ]:
# Step 6.5 (updated): semantic search helper + graph expansion using LangChain FAISS
def search_hybrid(query: str, top_k: int = 5) -> pd.DataFrame:
    """
    Use the LangChain FAISS vectorstore to search the hybrid text.
    Returns a DataFrame with: id, class_label, hybridText, score
    (score is the distance from FAISS; lower is better)
    """
    results = vectorstore.similarity_search_with_score(query, k=top_k)
    rows = []
    for doc, score in results:
        rows.append({
            "id": doc.metadata.get("id"),
            "class_label": doc.metadata.get("class_label"),
            "hybridText": doc.page_content,
            "score": score
        })
    return pd.DataFrame(rows)

def expand_in_graph(ids, depth: int = 2):
    """
    Given a list of Neo4j node IDs, fetch a small neighborhood.
    (Uses the 'driver' from earlier steps.)
    """
    with driver.session() as session:
        data = session.run(f"""
            MATCH (n:Resource)
            WHERE n.id IN $ids
            OPTIONAL MATCH p=(n)-[*1..{depth}]-(m:Resource)
            WITH n, collect(distinct m) AS nbrs
            RETURN n.id AS id,
                   n.rdfs_label AS label,
                   n.hasTicker AS ticker,
                   n.comment AS comment,
                   [x IN nbrs | coalesce(x.rdfs_label, x.id)] AS neighbors
            ORDER BY label
        """, {"ids": ids}).data()
    return data

In [ ]:
# Step 6.6: Example natural-language queries to test:
queries = [
    "equities regulated by the SEC",
    "Apple stock",
    "government bonds",
]

for q in queries:
    print(f"\nQuery: {q}")
    hits = search_hybrid(q, top_k=5)
    display(hits[["class_label","id","hybridText","score"]].head(5))
    # Expand top 3 in the graph
    top_ids = hits["id"].head(3).tolist()
    ctx = expand_in_graph(top_ids, depth=2)
    print("Graph context (top 3):")
    for row in ctx:
        print(f"- {row['label']} [{row.get('ticker')}] :: neighbors={row['neighbors'][:6]}")

In [ ]:
# Step 7.1: Helper: vector search + graph expansion (LangChain FAISS)
def vector_search(query: str, top_k: int = 5) -> pd.DataFrame:
    """
    Semantic search over the LangChain FAISS vectorstore built in Step 6.
    Returns a DataFrame with columns: id, class_label, hybridText, score
    """
    results = vectorstore.similarity_search_with_score(query, k=top_k)
    rows = []
    for doc, score in results:
        rows.append({
            "id": doc.metadata.get("id"),
            "class_label": doc.metadata.get("class_label"),
            "hybridText": doc.page_content,
            "score": float(score),
        })
    return pd.DataFrame(rows)

def graph_expand(ids: List[str], depth: int = 2) -> List[Dict[str, Any]]:
    """
    For each node id, collect a compact neighborhood and key properties.
    """
    with driver.session() as session:
        data = session.run(f"""
            MATCH (n:Resource)
            WHERE n.id IN $ids
            OPTIONAL MATCH p=(n)-[*1..{depth}]-(m:Resource)
            WITH n, collect(distinct m) AS nbrs
            RETURN n.id AS id,
                   n.rdfs_label AS label,
                   n.hasTicker AS ticker,
                   n.comment AS comment,
                   [x IN nbrs | coalesce(x.rdfs_label, x.id)] AS neighbors
            ORDER BY label
        """, {"ids": ids}).data()
    return data

def build_llm_context(query: str, top_k: int = 5, depth: int = 2, max_neighbors: int = 8) -> Dict[str, Any]:
    hits = vector_search(query, top_k=top_k)
    expanded = graph_expand(hits["id"].tolist(), depth=depth)
    exp_by_id = {row["id"]: row for row in expanded}
    
    # Clean URLs to entity names
    def clean(s):
        if "FinancialOntology/" in str(s):
            return str(s).split("/")[-1].replace("_", " ")
        return str(s)
    
    # Build Python dictionary format (Wu & Tsioutsiouliklis, 2024)
    relationships = {"type": {}, "issuedBy": {}, "isRegulatedBy": {}, "hasTicker": {}}
    citations = []
    
    for i, row in enumerate(hits.to_dict(orient="records"), start=1):
        meta = exp_by_id.get(row["id"], {})
        label = meta.get("label", row["id"].split("/")[-1])
        
        citations.append({"key": f"[E{i}]", "id": row["id"], "label": label})
        
        # Add properties
        if row.get("class_label"):
            relationships["type"][label] = row["class_label"]
        if meta.get("ticker"):
            relationships["hasTicker"][label] = meta["ticker"]
            
        # Parse relationships from hybrid text
        text = row.get("hybridText", "")
        if "issued by" in text.lower():
            issuer = text.split("issued by")[1].split()[0:2]
            issuer = " ".join(issuer).rstrip(".,")
            relationships["issuedBy"][label] = clean(issuer)
        if "regulated by" in text.lower():
            reg = text.split("regulated by")[1].split()[0]
            relationships["isRegulatedBy"][label] = clean(reg)
    
    # Format context with proper indentation
    relationships = {k: v for k, v in relationships.items() if v}
    rel_str = chr(10).join(f"    '{k}': {v}," for k, v in relationships.items())
    
    context = f"""Query: {query}

# Financial Knowledge Graph (Python Dictionary Format)
relationships = {{
{rel_str}
}}

# Usage: relationships['property']['entity'] returns the value
# Example: relationships['type']['AAPL'] returns 'Stock'"""
    
    return {"context": context, "hits": hits, "citations": citations}


In [ ]:
# Step 7.2: Example usage (show output)
query = "equities regulated by the SEC"
bundle = build_llm_context(query, top_k=5, depth=2, max_neighbors=8)

print(textwrap.dedent(bundle["context"]))
print("\nCitations:")
for c in bundle["citations"]:
    print(f"  {c['key']} -> {c['label']} ({c['id']})")


In [ ]:
# Step 7.3: Return a prompt template
chat_prompt = ChatPromptTemplate.from_template(
    "You are a financial assistant helping users understand investments and securities.\n"
    "Your approach to answering questions is to be short, direct, and concises.\n"
    "Take a conversational tone in responding to the question,"
    "but just state the facts and do not spend any extra text expanding on your answer.\n"
    "Use the relationships dictionary to answer questions accurately but conversationally.\n"
    "Important: Give clear, natural answers without mentioning URLs, technical details, or code.\n"
    "When citing sources, use the entity names naturally in your response.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n\n"
    "Provide a clear, conversational answer:"
)

In [ ]:
# Step 8.1: Demo queries
def make_context(question: str):
    bundle = build_llm_context(question, top_k=5, depth=2, max_neighbors=8)
    return {"context": bundle["context"], "question": question}

rag_chain = RunnableLambda(make_context) | chat_prompt | llm | StrOutputParser()

user_queries = [
   "Is AAPL a stock or bond?",
   "What type of instrument is USTB?",
   "Which authority regulates MSFT?",
   "Which equities are regulated by the SEC, and who issues them?",
   "What stocks do you know?  What bonds?"
]

for i, q in enumerate(user_queries, start=1):
    print(f"\n=== Query {i}: {q}")
    try:
        ans = rag_chain.invoke(q)
        print(ans)
    except Exception as e:
        print(f"[error] {e}")